In [1]:
import numpy as np
import os
import tqdm
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import Dataset
import torch.nn as nn

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
image_dir = '/kaggle/input/task-3/depth-estimation-challenge/competition-data/competition-data/training/images'
depths_dir = '/kaggle/input/task-3/depth-estimation-challenge/competition-data/competition-data/training/depths'

image_files = sorted(os.listdir(image_dir))
depths_files = sorted(os.listdir(depths_dir))

In [4]:
imgs = []
depths = []

for i in range(len(os.listdir('/kaggle/input/task-3/depth-estimation-challenge/competition-data/competition-data/training/images'))):
    imgs.append(cv2.imread(os.path.join(image_dir, image_files[i])))
    depths.append(cv2.imread(os.path.join(depths_dir, depths_files[i]), cv2.IMREAD_GRAYSCALE))

In [5]:
imgs_val_dir = '/kaggle/input/task-3/depth-estimation-challenge/competition-data/competition-data/validation/images'
depths_val_dir = '/kaggle/input/task-3/depth-estimation-challenge/competition-data/competition-data/validation/depths'

image_files_val = sorted(os.listdir(imgs_val_dir))
depths_files_val = sorted(os.listdir(depths_val_dir))

imgs_val = []
depths_val = []

for i in range(len(os.listdir('/kaggle/input/task-3/depth-estimation-challenge/competition-data/competition-data/validation/images'))):
    imgs_val.append(cv2.imread(os.path.join(imgs_val_dir, image_files_val[i])))
    depths_val.append(cv2.imread(os.path.join(depths_val_dir, depths_files_val[i]), cv2.IMREAD_GRAYSCALE))

In [6]:
imgs = np.array(imgs)
depths = np.array(depths)

In [7]:
imgs_val = np.array(imgs_val)
depths_val = np.array(depths_val)

In [8]:
depths_val = depths_val[:, :, :, np.newaxis]

In [9]:
depths = depths[:, :, :, np.newaxis]

In [10]:
data = [(transforms.ToTensor()(imgs[i]), transforms.ToTensor()(depths[i])) for i in range(len(imgs))]

In [11]:
data_val = [(transforms.ToTensor()(imgs_val[i]), transforms.ToTensor()(depths_val[i])) for i in range(len(imgs_val))]

In [12]:
train = DataLoader(data, batch_size=32, shuffle=True)
val = DataLoader(data_val, batch_size=4, shuffle=True)

In [13]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        # encoding
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        
        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)
        
        #  decoding
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(128, 64)
        self.d = nn.Conv2d(64, 1, kernel_size=1)
    
    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(nn.MaxPool2d(2)(enc1))
        enc3 = self.enc3(nn.MaxPool2d(2)(enc2))
        enc4 = self.enc4(nn.MaxPool2d(2)(enc3))
        
        bottleneck = self.bottleneck(nn.MaxPool2d(2)(enc4))
        
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.dec4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.dec3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.dec2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.dec1(dec1)
        
        return self.d(dec1)

In [14]:
model = UNet().to(device)

In [15]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [16]:
import copy
import torch.optim as optim

num_epochs = 70
best_val_loss = float('inf')
patience = 5
patience_counter = 0
best_model_weights = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for imgs, depths in train:
        imgs = imgs.to(device)
        depths = depths.to(device)
        outputs = model(imgs)
        optimizer.zero_grad()
        loss = criterion(outputs, depths)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, depths in val:
            imgs = imgs.to(device)
            depths = depths.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, depths)
            val_loss += loss.item()
    val_loss /= len(val)

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Check for improvement
    if val_loss < best_val_loss:
        print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best weights.")
        best_val_loss = val_loss
        patience_counter = 0
        best_model_weights = copy.deepcopy(model.state_dict())
    else:
        patience_counter += 1
        print(f"No improvement for {patience_counter} epochs.")

    if patience_counter >= patience:
        print("Early stopping due to no improvement in validation loss.")
        break

    scheduler.step()

model.load_state_dict(best_model_weights)


Epoch 1/70, Train Loss: 0.0610, Validation Loss: 0.0532
Validation loss improved from inf to 0.0532. Saving best weights.
Epoch 2/70, Train Loss: 0.2861, Validation Loss: 0.0439
Validation loss improved from 0.0532 to 0.0439. Saving best weights.
Epoch 3/70, Train Loss: 0.0274, Validation Loss: 0.0203
Validation loss improved from 0.0439 to 0.0203. Saving best weights.
Epoch 4/70, Train Loss: 0.0186, Validation Loss: 0.0175
Validation loss improved from 0.0203 to 0.0175. Saving best weights.
Epoch 5/70, Train Loss: 0.0170, Validation Loss: 0.0165
Validation loss improved from 0.0175 to 0.0165. Saving best weights.
Epoch 6/70, Train Loss: 0.0163, Validation Loss: 0.0155
Validation loss improved from 0.0165 to 0.0155. Saving best weights.
Epoch 7/70, Train Loss: 0.0152, Validation Loss: 0.0155
Validation loss improved from 0.0155 to 0.0155. Saving best weights.
Epoch 8/70, Train Loss: 0.0145, Validation Loss: 0.0141
Validation loss improved from 0.0155 to 0.0141. Saving best weights.
Epo

<All keys matched successfully>

In [17]:
image_dir_test = '/kaggle/input/task-3/depth-estimation-challenge/competition-data/competition-data/testing/images'
image_files_test = sorted(os.listdir(image_dir_test))

imgs_test = []

for i in range(len(os.listdir(image_dir_test))):
    imgs_test.append(cv2.imread(os.path.join(image_dir_test, image_files_test[i])))

imgs_test = np.array(imgs_test)

In [18]:
data = [transforms.ToTensor()(imgs_test[i]) for i in range(len(imgs_test))]

In [19]:
test = DataLoader(data, batch_size=4, shuffle=False)

In [20]:
preds = []
with torch.no_grad():
    for imgs in test:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds.extend(outputs.cpu().numpy())

In [21]:
preds = np.array(preds)

In [22]:
preds.shape

(836, 1, 256, 256)

In [23]:
preds = (preds * 255).astype(np.uint8)

In [24]:
output_dir = "output_images"
os.makedirs(output_dir, exist_ok=True)

for idx, img_array in enumerate(preds):
    img = img_array.squeeze()
    filename = os.path.join(output_dir, f"image_{idx}.png")
    cv2.imwrite(filename, img)

In [25]:
import zipfile

folder_to_zip = "output_images"
zip_filename = "zipped_folder.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_to_zip):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, folder_to_zip)
            zipf.write(file_path, arcname)